import cv2

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
video_capture = cv2.VideoCapture(0)

def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
    return faces

face_id = input("Enter the ID of the person: ")  # Input ID for the person whose face is being captured
count = 0  # Initialize a counter for the number of images captured

while True:
    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully

    faces = detect_bounding_box(video_frame)  # apply face detection to the video frame

    # Capture and save images of detected faces
    for (x, y, w, h) in faces:
        count += 1
        face_image = video_frame[y:y+h, x:x+w]  # extract the detected face region
        cv2.imwrite(f"dataset/person_{face_id}_{count}.jpg", face_image)  # save the face image
        cv2.putText(video_frame, f"Image {count}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    cv2.imshow("My Face Detection Project", video_frame)  # display the processed frame

    if cv2.waitKey(1) & 0xFF == ord("q") or count >= 30:  # break the loop if 'q' is pressed or 30 images are captured
        break

video_capture.release()
cv2.destroyAllWindows()


import cv2

def capture_images(output_folder, person_id, num_images=30):
    # Create a VideoCapture object to access the webcam
    video_capture = cv2.VideoCapture(0)

    # Create a Haar cascade classifier for face detection
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    count = 0  # Initialize counter for captured images

    while count < num_images:
        # Capture frame-by-frame from the webcam
        ret, frame = video_capture.read()

        # Convert the frame to grayscale for face detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        # Draw rectangles around the faces and save the images
        for (x, y, w, h) in faces:
            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Increment count and save the captured image
            count += 1
            image_name = f"person_{person_id}_{count}.jpg"
            cv2.imwrite(output_folder + '/' + image_name, frame[y:y+h, x:x+w])

            # Display the image count
            cv2.putText(frame, f"Image {count}/{num_images}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Display the frame
        cv2.imshow('Capture Images', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture object and close all windows
    video_capture.release()
    cv2.destroyAllWindows()

# Set the output folder where images will be saved
output_folder = 'dataset'

# Input the person's ID
person_id = input("Enter the ID of the person: ")

# Capture images and save them to the dataset folder
capture_images(output_folder, person_id)


import cv2

# Initialize face detector
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture
video_capture = cv2.VideoCapture(0)

# Initialize face recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Initialize user data
user_data = {}

def capture_data(face_id, user_name, face_image):
    # Save face image with user_id as filename
    cv2.imwrite(f"dataset/{face_id}.jpg", face_image)
    # Update user data dictionary
    user_data[face_id] = user_name

def train_model():
    # Load images and labels for training
    faces = []
    labels = []
    for face_id, user_name in user_data.items():
        face_image = cv2.imread(f"dataset/{face_id}.jpg", cv2.IMREAD_GRAYSCALE)
        faces.append(face_image)
        labels.append(int(face_id))
    # Train the recognizer
    recognizer.train(faces, np.array(labels))

def detect_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def recognize_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detect_faces(frame)
    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y+h, x:x+w]
        # Predict the face
        face_id, confidence = recognizer.predict(face_roi)
        if confidence < 100:
            user_name = user_data.get(face_id, "Unknown")
            cv2.putText(frame, user_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        else:
            cv2.putText(frame, "Unknown", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    return frame

def main():
    while True:
        ret, frame = video_capture.read()
        if ret == False:
            break
        faces = detect_faces(frame)
        for (x, y, w, h) in faces:
            # Capture user data when a face is detected
            user_name = input("Enter your name: ")
            face_image = frame[y:y+h, x:x+w]
            capture_data(len(user_data)+1, user_name, cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY))
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, user_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        frame = recognize_faces(frame)
        cv2.imshow('Face Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Train the model with captured data
    train_model()
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [8]:
pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/232.6 kB 325.1 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/232.6 kB 325.1 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/232.6 kB 217.9 kB/s eta 0:00:01
   ------------- ------------------------- 81.9/232.6 kB 327.3 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/232.6 kB 327.7 kB/s eta 0:00:01
   -------------------- ----------------- 122.9/232.6 kB 379.3 kB/s eta 0:00:01
   -------------------------- ----------- 163.8/232.6 kB 446.5 kB/s eta 0:00:01
   --------------------------

In [2]:
import cv2
import numpy as np
import os


In [3]:
import PyPDF2
from pdf2image import convert_from_path

ModuleNotFoundError: No module named 'pdf2image'

In [1]:


# Import face recognition module from cv2.face
from cv2.face import LBPHFaceRecognizer_create

# Initialize face detector
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize face recognizer
recognizer = LBPHFaceRecognizer_create()

# Initialize user data
user_data = {}

def capture_data(face_id, user_name, face_image):
    # Save face image with user_id as filename
    cv2.imwrite(f"dataset/{face_id}.jpg", face_image)
    # Update user data dictionary
    user_data[face_id] = user_name

def train_model():
    # Load images and labels for training
    faces = []
    labels = []
    for face_id, user_name in user_data.items():
        face_image = cv2.imread(f"dataset/{face_id}.jpg", cv2.IMREAD_GRAYSCALE)
        faces.append(face_image)
        labels.append(int(face_id))
    # Train the recognizer
    recognizer.train(faces, np.array(labels))

def detect_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def recognize_faces(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detect_faces(frame)
    for (x, y, w, h) in faces:
        face_roi = gray_frame[y:y+h, x:x+w]
        # Predict the face
        face_id, confidence = recognizer.predict(face_roi)
        if confidence < 100:
            user_name = user_data.get(face_id, "Unknown")
            cv2.putText(frame, user_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        else:
            cv2.putText(frame, "Unknown", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    return frame

def process_images_in_folder(folder_path):
    # Process all images in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            frame = cv2.imread(image_path)
            frame = recognize_faces(frame)
            cv2.imshow('Face Recognition', frame)
            cv2.waitKey(0)
    cv2.destroyAllWindows()

def process_video(video_path):
    # Process video file
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = recognize_faces(frame)
        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

def extract_images_from_pdf(pdf_path):
    # Extract images from PDF file
    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        frame = np.array(image)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = recognize_faces(frame)
        cv2.imshow('Face Recognition', frame)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

def main():
    # Capture data from images
    process_images_in_folder('images_folder')

    # Train the model with captured data
    train_model()

    # Process video file
    process_video('video_file.mp4')

    # Extract images from PDF file
    extract_images_from_pdf('pdf_file.pdf')

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'pdf2image'